Formação Cientista de Dados II - Professor Fernando Amaral
Auto ML

In [12]:
#!pip install h2o

# Imports

- **H2O**: Framework de IA que possui vários modelos de DL e ML, além do Auto ML que vamos utilizar também.


In [13]:
import h2o
import pandas as pd

from h2o.automl import H2OAutoML
from h2o.estimators import H2ORandomForestEstimator

# Carregar a base

In [14]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 min 36 secs
H2O_cluster_timezone:,America/Fortaleza
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.4
H2O_cluster_version_age:,1 month and 8 days
H2O_cluster_name:,H2O_from_python_samuelhericlis_xp86em
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.535 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [15]:
#Importa dados 
imp = pd.read_csv('/Users/samuelhericlis/Desktop/IBM/cursos/formacao_cientista_de_dados_II_topicos_avançados/CIENTISTADADOS2/Dados/9.AutoML/Churn_treino.csv', sep=";")
display(imp.head(3))
imp.info()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0,1,1,1,10134888,1
1,608,Spain,Female,41,1,8380786,1,0,1,11254258,0
2,502,France,Female,42,8,1596608,3,1,0,11393157,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CreditScore      10000 non-null  int64 
 1   Geography        10000 non-null  object
 2   Gender           10000 non-null  object
 3   Age              10000 non-null  int64 
 4   Tenure           10000 non-null  int64 
 5   Balance          10000 non-null  int64 
 6   NumOfProducts    10000 non-null  int64 
 7   HasCrCard        10000 non-null  int64 
 8   IsActiveMember   10000 non-null  int64 
 9   EstimatedSalary  10000 non-null  int64 
 10  Exited           10000 non-null  int64 
dtypes: int64(9), object(2)
memory usage: 859.5+ KB


# Instanciar o ambiente

In [16]:
# divide em treino e teste
imp = h2o.H2OFrame(imp)
treino,teste = imp.split_frame(ratios=[.7])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [17]:
#Transforma a variável dependente em fator
treino["Exited"] = treino["Exited"].asfactor()
teste["Exited"] = teste["Exited"].asfactor()

## Auto ML

Busca o modelo por 180 segundos, podemos em vez disso definir max_models. Ele segue a AUC como medida de peformance.


In [18]:
modelo_automl = H2OAutoML(max_runtime_secs=180, sort_metric = 'AUC')
modelo_automl.train( y="Exited", training_frame=treino)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_4_AutoML_2_20220911_181427

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0769661028053544
RMSE: 0.27742765328163377
LogLoss: 0.25703732634752724
Null degrees of freedom: 6943
Residual degrees of freedom: 6938
Null deviance: 7016.160670224121
Residual deviance: 3569.7343883144586
AIC: 3581.7343883144586
AUC: 0.9339899553478606
AUCPR: 0.8282977401204199
Gini: 0.8679799106957211

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.28745115328208326: 


,,0,1,Error,Rate
0,0,5072.0,459.0,0.083,(459.0/5531.0)
1,1,319.0,1094.0,0.2258,(319.0/1413.0)
2,Total,5391.0,1553.0,0.112,(778.0/6944.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.287451,0.737694,211.0
1,max f2,0.188431,0.790250,255.0
2,max f0point5,0.544774,0.785787,127.0
3,max accuracy,0.504692,0.895593,138.0
4,max precision,0.995650,1.000000,0.0
5,max recall,0.019271,1.000000,381.0
6,max specificity,0.995650,1.000000,0.0
7,max absolute_mcc,0.331829,0.668956,193.0
8,max min_per_class_accuracy,0.205943,0.852287,247.0
9,max mean_per_class_accuracy,0.188431,0.853598,255.0



Gains/Lift Table: Avg response rate: 20,35 %, avg score: 20,21 %


...


Gains/Lift Table: Avg response rate: 20,35 %, avg score: 20,35 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010081,0.963235,4.844161,4.844161,0.985714,0.978604,0.985714,0.978604,0.048832,0.048832,384.416136,384.416136,0.048651
1,2,0.020017,0.933326,4.771921,4.808301,0.971014,0.948228,0.978417,0.963525,0.047417,0.096249,377.192119,380.830113,0.095707
2,3,0.030098,0.906480,4.563340,4.726257,0.928571,0.920179,0.961722,0.949007,0.046001,0.142251,356.334041,372.625687,0.140804
3,4,0.040035,0.876372,4.344585,4.631525,0.884058,0.889482,0.942446,0.934233,0.043171,0.185421,334.458496,363.152535,0.182528
4,5,0.050115,0.830920,4.071904,4.518958,0.828571,0.852395,0.919540,0.917771,0.041047,0.226469,307.190375,351.895779,0.221406
5,6,0.100086,0.624815,3.583097,4.051701,0.729107,0.736162,0.824460,0.827097,0.179052,0.405520,258.309726,305.170080,0.383463
6,7,0.150058,0.449068,2.577564,3.560793,0.524496,0.527251,0.724568,0.727245,0.128804,0.534324,157.756404,256.079345,0.482435
7,8,0.200029,0.324229,1.798630,3.120570,0.365994,0.381191,0.634989,0.640794,0.089880,0.624204,79.862985,212.056972,0.532539
8,9,0.299971,0.201319,1.295863,2.512626,0.263689,0.256048,0.511282,0.512607,0.129512,0.753715,29.586324,151.262622,0.569662
9,10,0.400058,0.136541,0.848524,2.096301,0.172662,0.165506,0.426566,0.425769,0.084926,0.838641,-15.147627,109.630109,0.550628




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.859154,0.004155,0.856346,0.855378,0.865018,0.857038,0.861993
1,auc,0.869317,0.011272,0.855137,0.862139,0.879428,0.881654,0.868228
2,err,0.140846,0.004155,0.143654,0.144622,0.134982,0.142962,0.138007
3,err_count,195.600000,7.334848,206.000000,199.000000,191.000000,195.000000,187.000000
4,f0point5,0.655323,0.020623,0.659341,0.647510,0.687978,0.632490,0.649297
5,f1,0.642316,0.023966,0.619926,0.629423,0.674617,0.660870,0.626747
6,f2,0.631333,0.044060,0.584958,0.612319,0.661765,0.691916,0.605710
7,lift_top_group,4.854020,0.260780,4.812080,4.547026,4.669967,5.127820,5.113207
8,logloss,0.330654,0.013407,0.349752,0.335966,0.325648,0.313445,0.328462
9,max_per_class_error,0.374923,0.059472,0.436242,0.398577,0.346535,0.285714,0.407547



See the whole table with table.as_data_frame()


executar o modelo, o Random Forest.

In [19]:
model_floresta = H2ORandomForestEstimator(nfolds=5)
model_floresta.train( y="Exited", training_frame=treino)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1662930769104_9762


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,546236.0,19.0,20.0,19.96,794.0,931.0,864.68




ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.11067903204081551
RMSE: 0.3326845834132016
LogLoss: 0.7161971343411185
Mean Per-Class Error: 0.24009209879642543
AUC: 0.8326434048686275
AUCPR: 0.652133768071832
Gini: 0.6652868097372551

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.34782608695652173: 


,,0,1,Error,Rate
0,0,4891.0,640.0,0.1157,(640.0/5531.0)
1,1,515.0,898.0,0.3645,(515.0/1413.0)
2,Total,5406.0,1538.0,0.1663,(1155.0/6944.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.841663,0.015856,0.849702,0.842253,0.855964,0.814815,0.845578
1,auc,0.845873,0.018120,0.843873,0.865922,0.862925,0.827792,0.828854
2,err,0.158337,0.015856,0.150298,0.157746,0.144036,0.185185,0.154422
3,err_count,220.000000,24.484690,202.000000,224.000000,192.000000,255.000000,227.000000
4,f0point5,0.611432,0.029612,0.580929,0.615732,0.643357,0.580975,0.636169
5,f1,0.617951,0.026484,0.589431,0.630363,0.657143,0.602184,0.610635
6,f2,0.625499,0.034462,0.598185,0.645707,0.671533,0.625000,0.587071
7,lift_top_group,4.718936,0.227594,4.941176,4.879725,4.690185,4.356905,4.726688
8,logloss,0.436580,0.073549,0.429626,0.356310,0.382413,0.473982,0.540567
9,max_per_class_error,0.368890,0.043172,0.395833,0.343643,0.318519,0.358804,0.427653



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-09-11 18:17:33,3.524 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2022-09-11 18:17:33,3.553 sec,1.0,0.457927,7.237607,0.681840,0.380930,2.387771,0.209524
2,,2022-09-11 18:17:33,3.570 sec,2.0,0.443549,6.385947,0.692595,0.401797,2.545056,0.220620
3,,2022-09-11 18:17:33,3.583 sec,3.0,0.432288,5.731194,0.704419,0.418338,2.635671,0.219797
4,,2022-09-11 18:17:33,3.603 sec,4.0,0.421652,5.074907,0.713462,0.425014,2.690616,0.220526
5,,2022-09-11 18:17:33,3.615 sec,5.0,0.411477,4.494686,0.725851,0.447362,2.870451,0.218182
6,,2022-09-11 18:17:33,3.627 sec,6.0,0.404639,4.062987,0.735579,0.461744,2.966391,0.214055
7,,2022-09-11 18:17:33,3.640 sec,7.0,0.397019,3.652881,0.743774,0.477916,3.110453,0.212194
8,,2022-09-11 18:17:34,3.662 sec,8.0,0.387405,3.240329,0.752254,0.500531,3.323686,0.204885
9,,2022-09-11 18:17:34,3.673 sec,9.0,0.379811,2.863543,0.760789,0.519807,3.514289,0.205034



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Age,9572.720703,1.000000,0.260667
1,NumOfProducts,5387.621582,0.562810,0.146706
2,Balance,4700.612793,0.491043,0.127999
3,EstimatedSalary,4655.495117,0.486329,0.126770
4,CreditScore,4624.392090,0.483080,0.125923
5,Tenure,3144.170898,0.328451,0.085616
6,Geography,1632.934326,0.170582,0.044465
7,IsActiveMember,1512.424561,0.157993,0.041184
8,HasCrCard,748.123108,0.078152,0.020372
9,Gender,745.417175,0.077869,0.020298


## Ranking dos melhores modelos no AutoML

In [20]:
ranking = modelo_automl.leaderboard
ranking = ranking.as_data_frame()
ranking

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
0,StackedEnsemble_BestOfFamily_4_AutoML_2_20220911_181427,0.868941,0.330822,0.705922,0.225520,0.317177,0.100601
1,StackedEnsemble_AllModels_3_AutoML_2_20220911_181427,0.868690,0.331134,0.705979,0.231577,0.317416,0.100753
2,StackedEnsemble_AllModels_2_AutoML_2_20220911_181427,0.867166,0.332332,0.704268,0.225468,0.317815,0.101006
3,StackedEnsemble_BestOfFamily_3_AutoML_2_20220911_181427,0.867089,0.332531,0.704112,0.223112,0.317968,0.101104
4,XGBoost_grid_1_AutoML_2_20220911_181427_model_9,0.865878,0.333930,0.701372,0.224595,0.318735,0.101592
...,...,...,...,...,...,...,...
80,DeepLearning_grid_2_AutoML_2_20220911_181427_model_1,0.825932,0.405256,0.634256,0.267401,0.341244,0.116447
81,DeepLearning_grid_1_AutoML_2_20220911_181427_model_1,0.823837,0.396760,0.639657,0.248231,0.340856,0.116183
82,GBM_grid_1_AutoML_2_20220911_181427_model_23,0.821465,0.387418,0.604944,0.258581,0.344083,0.118393
83,DeepLearning_grid_1_AutoML_2_20220911_181427_model_2,0.811961,0.462325,0.612123,0.280747,0.357836,0.128046


## Peformance no geral

In [21]:
performance = model_floresta.model_performance(test_data=teste)
performance


ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.10826651221981752
RMSE: 0.32903877008616705
LogLoss: 0.4354563021424353
Mean Per-Class Error: 0.24403782894736842
AUC: 0.8434706056005398
AUCPR: 0.6767867630551895
Gini: 0.6869412112010795

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.35: 


,,0,1,Error,Rate
0,0,2157.0,275.0,0.1131,(275.0/2432.0)
1,1,234.0,390.0,0.375,(234.0/624.0)
2,Total,2391.0,665.0,0.1666,(509.0/3056.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.350000,0.605120,130.0
1,max f2,0.160000,0.685654,257.0
2,max f0point5,0.525000,0.676796,63.0
3,max accuracy,0.525000,0.861584,63.0
4,max precision,1.000000,1.000000,0.0
5,max recall,0.000000,1.000000,383.0
6,max specificity,1.000000,1.000000,0.0
7,max absolute_mcc,0.520000,0.525360,66.0
8,max min_per_class_accuracy,0.202595,0.753205,230.0
9,max mean_per_class_accuracy,0.282595,0.763137,171.0



Gains/Lift Table: Avg response rate: 20,42 %, avg score: 21,39 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.012435,0.940000,4.768556,4.768556,0.973684,0.954956,0.973684,0.954956,0.059295,0.059295,376.855601,376.855601,0.058884
1,2,0.025851,0.900000,4.777986,4.773450,0.975610,0.908293,0.974684,0.930738,0.064103,0.123397,377.798624,377.345018,0.122575
2,3,0.030759,0.880000,4.570940,4.741135,0.933333,0.880000,0.968085,0.922642,0.022436,0.145833,357.094017,374.113475,0.144600
3,4,0.042866,0.840000,4.367983,4.635741,0.891892,0.854144,0.946565,0.903295,0.052885,0.198718,336.798337,363.574085,0.195840
4,5,0.051702,0.800000,3.809117,4.494482,0.777778,0.808519,0.917722,0.887099,0.033654,0.232372,280.911681,349.448231,0.227026
5,6,0.102421,0.600000,3.317618,3.911690,0.677419,0.687666,0.798722,0.788338,0.168269,0.400641,231.761787,291.169001,0.374736
6,7,0.150196,0.466500,2.448718,3.446344,0.500000,0.528561,0.703704,0.705708,0.116987,0.517628,144.871795,244.634378,0.461707
7,8,0.203861,0.380000,1.552846,2.947895,0.317073,0.420211,0.601926,0.630553,0.083333,0.600962,55.284553,194.789480,0.498988
8,9,0.300065,0.251190,1.116082,2.360596,0.227891,0.308361,0.482007,0.527255,0.107372,0.708333,11.608233,136.059615,0.513021
9,10,0.424084,0.160000,1.007916,1.965021,0.205805,0.196426,0.401235,0.430508,0.125000,0.833333,0.791557,96.502058,0.514254


## Prever com o melhor modelo do AutoML


In [22]:
teste = pd.read_csv('/Users/samuelhericlis/Desktop/IBM/cursos/formacao_cientista_de_dados_II_topicos_avançados/CIENTISTADADOS2/Dados/9.AutoML/Churn_prever.csv', sep=";")
teste = h2o.H2OFrame(teste)
prever = modelo_automl.leader.predict(teste)
prever

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.919968,0.0800318
0,0.924275,0.0757249
1,0.00628888,0.993711
0,0.989077,0.0109235
0,0.935412,0.0645876
0,0.900644,0.0993557
0,0.723315,0.276685
0,0.839489,0.160511
0,0.903602,0.0963984
